# 3d mitochondria visualizer using vtk
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


# Enter a cellid of interest

In [6]:
# pull all mitos from a cellid of interest
cellid = 648518346349535074

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf

# interesting cellids
# 648518346349530724 microglia with 149 mitos
# 648518346349527319 astrocyte containing the largest mito in the volume and several additional large mitos
# 648518346349537555 apical dendrite likely from a L5 or L6 pyr neuron
# 648518346349537741 pyr neuron with largest mito in the volume for a neuron
# 648518346349508279 oligodendrocyte
# 648518346349525715 astrocyte
# 648518346342795947 astrocyte contacting two blood vessels
# 648518346349535074 pyr neuron (likely Layer 3)
# 648518346349538053 pyr neuron from clean 65

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
1659,1615870,553776,71126,74242,502,70930,73286,489,71430,75370,520,648518346349535074,254631.08,265786.36,20080
2312,1568426,1112128,71096,55028,2003,69934,54560,1847,72248,55480,2157,648518346349535074,254523.68,197000.24,80120
2726,2064781,568704,79566,67434,679,79072,67212,648,79878,67544,717,648518346349535074,284846.28,241413.72,27160
2788,2064748,507864,79248,66466,573,79038,66000,513,79524,67054,634,648518346349535074,283707.84,237948.28,22920
3990,1004000,5138632,63832,59358,801,60026,55072,662,69494,63488,1112,648518346349535074,228518.56,212501.64,32040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402961,2182839,595472,80638,66458,583,80318,66102,565,81198,66810,620,648518346349535074,288684.04,237919.64,23320
2403632,2418834,617440,84172,66976,662,83496,66806,607,85002,67190,703,648518346349535074,301335.76,239774.08,26480
2404268,2064321,641648,78878,66462,631,78622,65786,571,79110,66998,676,648518346349535074,282383.24,237933.96,25240
2405624,2185958,703632,79514,67546,772,79238,67440,707,80148,67654,810,648518346349535074,284660.12,241814.68,30880


In [7]:
mito_query_mitolist = mito_querydf.mito_id.to_list()
print(f"length: "+str(len(mito_query_mitolist)))
print(f"type: "+str(type(mito_query_mitolist)))
print('')
# print(mito_query_mitolist) # uncomment to print the mito id list

length: 518
type: <class 'list'>



# vtk 3d viewer

In [8]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [9]:
cell_id = cellid

In [10]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [11]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [12]:
len(mito_query_mitolist)

518

In [13]:
mito_query_mitolist[0]

1615870

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [14]:
# download the mito meshes for this cell

for i in range(len(mito_query_mitolist)):
    mito_id = mito_query_mitolist[i]
    mito_seg_id = mito_id
#    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [15]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator = {}
for i in range(len(mito_query_mitolist)):
    mito_seg_id = mito_query_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator)

In [16]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.075 # 0.075
cell_color = (0.7, 0.7, 0.7) # (0.2, 0.4, 0.7) blue
mito_opac = 0.35 # 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002A9433D23A0) at 0x000002A96C2CCBE8>

In [17]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

# trimesh_vtk.render_actors(actor_iterator.values(),
#                           filename='vtk_images/'+str(cellid)+'.png',
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )

In [18]:
type(mesh_actor)

vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor

In [19]:
actor_iterator

{'mito_mesh_actor_0': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597BC3D0) at 0x000002A96C294528>,
 'mito_mesh_actor_1': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597BC830) at 0x000002A96C294588>,
 'mito_mesh_actor_2': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B95E0) at 0x000002A96C2945E8>,
 'mito_mesh_actor_3': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B3E60) at 0x000002A96C294648>,
 'mito_mesh_actor_4': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B6C50) at 0x000002A96C2946A8>,
 'mito_mesh_actor_5': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B35A0) at 0x000002A96C294708>,
 'mito_mesh_actor_6': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B0350) at 0x000002A96C294768>,
 'mito_mesh_actor_7': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B4FE0) at 0x000002A96C2947C8>,
 'mito_mesh_actor_8': <vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000002A9597B1FC0) at 0x00

In [20]:
type(mito_mesh_actor_0)

vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor

In [21]:
mesh

<trimesh.Mesh(vertices.shape=(1272818, 3), faces.shape=(2543544, 3))>